In [31]:
matlab_path = '''matlab'''
# This assumes that Matlab is in your path. If not then direct it to
# program files / matlab / version/ bin/ matlab.exe
# needs to be in triple quotes


matlab_cmd = '''"cd('C:\github\perla_tonetti_waugh\code\calibration');calibrate_wrap;"'''
# then the way this works is you have to give the matlab command to change the working directory
# this is the cd part; then call the function which in this case is calibrate_wrap
# needs to be in triple quotes 

In [34]:
!{matlab_path} -batch {matlab_cmd}

# Then the ! command passes the stuff to the command prompt
# or terminal. the {} remove the triple quotes
# -batch runs this stuff in the background, i.e. no matlab
# terminal is opend, it will autormatically exit and close matlab
# then the next {} is function call to matlab

Calibration Targets...
Real Rate and Productivity Growth
    0.0283    0.0079

BEJK Exporter Moments: Fraction of Exporters, Relative Size
    0.0330    4.8000

Home Trade Share
    0.8937

Entry Moment
    0.0200

Firm Moments
    0.2600    0.2900    0.2850    0.1610
    0.0940    0.1360    0.2700    0.5010

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 
Optimization terminated:
 the current x satisfies the termination criteria using OPTIONS.TolX of 1.000000e-04 
 and F(X) satisfies the convergence criteria using OPTIONS.TolFun of 1.000000e-04 

Elapsed time is 152.554542 seconds.
    0.0037

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Calibration Results
Parameter Values
d, theta, kappa, 1/chi, mu, upsilon, sigma, delta, rho
    3.0225    4.9890    0.1042    7.8835   -0.0311    0.0483    3.1669    0.0200    0.0203

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Moments: Model
Calibration Targe

---
### <a name="codeappendix"></a>  Matlab Code Appendix

#### <a name="cal_func"></a> Structure of main calibration file [``calibration_wrap.m``](calibration_wrap.m)

The file [``calibration_wrap.m``](calibration_wrap.m) calibrates the model and computes steady state to steady state results. Transition dynamics are all in julia and described [here]().

The file ``calibration_wrap.m`` does the following:

**1.** Reads in the moments that are generated from the python notebook described above in [create calibration moments](#calibration)

**2.** Minimizes the distance between the data moments and model moments. Code computing the model moments is organized in the following way:

  - [``calibrate_growth.m``](calibrate_growth.m) organizes parameters, passes them to the function to retrieve model moments, then constructs the objective function the calibration is minimizing. The moments are stacked in the following order:

    - g, lambda_{ii}, fraction of exporters, size of exporters vs non exporters, and then the quantile moments (note how these are passed through) into vector form.

    - Then line 47 describes the exact objective function used. This is just the standard GMM objective with an identity matrix. It looks like I scaled the diagonals to be 100, this should not matter.


  -  <a name="main_file"></a> [``compute_growth_fun_cal.m``](compute_growth_fun_cal.m) takes in parameters, compute equilibrium, the moments, and welfare. The following functions perform these operations.

    - [``eq_functions/calculate_equilibrium.m``](eq_functions/calculate_equilibrium.m) finds an equilibrium.

    - [``eq_functions/selection_gbm_constant_Theta_functions.m``](eq_functions/selection_gbm_constant_Theta_functions.m) are the equilibrium functions which map to equations in the paper. **This is the one mathematica generated file**

    - [``static_firm_moments.m``](static_firm_moments.m) Computes the BEJK moments, fraction of exporters and then the relative size of domestic shipments for exporters vs non-exporters.

    - [``markov_chain/generate_transition_matrix.m``](markov_chain/generate_transition_matrix.m) constructs the transition matrix across productivity states. **Note** that the code [``markov_chain/generate_stencils.m``](markov_chain/generate_stencils.m) is required to generate the grid. This is called before the calibration routine since it is kept constant.

    - [``markov_chain/coarsen_to_quintiles_four.m``](markov_chain/coarsen_to_quintiles_four.m) maps the transition matrix into a four by four matrix over a 5 year horizon to match up with data moments.

**3.** Given the calibration results, several things happen:

- A file [``output/main_results/calibration_params.csv``](output/main_results/calibration_params.csv) is created with the calibrated parameters. These are in the following order: d, theta, kappa, 1/chi, mu, upsilon, sigma, delta, rho.

- An associated ``.mat`` file ``cal_params.mat`` is generated for use within matlab. The variable ``new_cal`` has the values in the in the following order: d, theta, kappa, 1/chi, mu, upsilon, sigma.

- Across steady state results are presented. To compute across steady state results the code [``compute_growth_fun_cal``](compute_growth_fun_cal.m) is called with the appropriately changed parameter values.
